### Download a Growing Degree Day file

This notebook demonstrates how to access and download the previous day's Growing Degree Day file to CSV

##### Import Dependencies

In [ ]:
import pandas as pd
import requests
from io import StringIO
import arcgis
from arcgis.gis import GIS
from datetime import datetime, timedelta

##### Set the ArcGIS Portal to PDI-Development

In [ ]:
pdi = GIS(url="https://pdi-development.scinet.usda.gov/portal")
print("ArcGIS portal set to " + pdi.url)

##### Set the Desired Growing Degree Day date

In [ ]:
# Get yesterday's date
date = (datetime.today() - timedelta(days=1)).strftime("%Y%m%d")
date

##### Query the portal for the item

In [ ]:
csv_item = pdi.content.search(query=f'growing-degree-day-{date} type:CSV')[0]
csv_item

##### Download the item to a CSV file

In [ ]:
csv_url = f"https://pdi-development.scinet.usda.gov/portal/sharing/rest/content/items/{csv_item.id}/data"
response = requests.get(csv_url)
if response.status_code == 200:
    csv_data = StringIO(response.text)
    df = pd.read_csv(csv_data)
    df = df.drop(['fid', 'x', 'y'], axis=1)
    for col in df.columns:
        if df[col].dtype == 'float64':
            df[col] = round(df[col],2)
    df.to_csv(f'growing-degree-day-{date}.csv', escapechar='"', index=False)
else:
    print(f"Failed to download file. Status code: {response.status_code}")